In [ ]:
# !pip install r5py

In [ ]:
# !pip install pyrosm
# !pip install folium
# !pip install geojson


In [8]:
import geopandas as gpd
import pandas as pd

import datetime
import time 
import r5py
import os

### 1. Building transportation network using OSM and GTFS data

In [10]:

# Ensure the file paths are correct
OSM_path = "/Users/max/Desktop/Transit_Dashboard/data/OSM_data/Toronto.osm.pbf" #"../../data/OSM_data/Toronto.osm.pbf"
GTFS_path = "/Users/max/Desktop/Transit_Dashboard/data/GTFS_data/raw/latest_feed_version_2024-10-22.zip"  #"../../data/GTFS_data/raw/latest_feed_version_2024-10-22.zip"


# Make the R5 transport network using OSM and GTFS 
network_start = time.time()
transport_network = r5py.TransportNetwork(OSM_path, [GTFS_path])
network_total_time = time.time()-network_start
print("Transportion network building time", network_total_time, "seconds." )

transport_modes = [
    r5py.TransportMode.TRANSIT,
    r5py.TransportMode.WALK,
]

Transportion network building time 87.78173804283142 seconds.


### 2. Load Origins and Destinations for travel time calculation.

In [13]:
import geopandas as gpd
import time
import r5py
import datetime

def compute_travel_time_matrix(origins_file, destinations_file, transport_network, output_file, origin_id_col='', destination_id_col=''):
    # Load the GeoJSON files
    origins = gpd.read_file(origins_file)
    destinations = gpd.read_file(destinations_file)
    
    # Check if the origin ID column exists, if not, raise an error or rename it
    if "id" not in origins.columns:
        if origin_id_col not in origins.columns:
            raise ValueError(f"Origin ID column '{origin_id_col}' not found in the origins file.")
        origins['id'] = origins[origin_id_col]
    
    # Check if the destination ID column exists, if not, raise an error or rename it
    if "id" not in destinations.columns:
        if destination_id_col not in destinations.columns:
            raise ValueError(f"Destination ID column '{destination_id_col}' not found in the destinations file.")
        destinations['id'] = destinations[destination_id_col]
    
    # Start timer
    start = time.time()
    
    # Compute travel time matrix
    travel_time_matrix = r5py.TravelTimeMatrixComputer(
        transport_network,
        origins=origins,
        destinations=destinations,
        transport_modes=[r5py.TransportMode.TRANSIT],
        # Optional: specify the departure time if needed
        # departure=datetime.datetime(2019, 5, 13, 14, 0, 0),
    ).compute_travel_times()
    
    # End timer
    end = time.time()
    running_time = end - start
    print(f"Running time: {running_time} seconds")
    
    # Save the results to a CSV file
    travel_time_matrix.to_csv(output_file)

# Census tract to census tract
compute_travel_time_matrix(
    origins_file="/Users/max/Desktop/Transit_Dashboard/data/census_tract_data/toronto_ct_centroids1.geojson",
    destinations_file="/Users/max/Desktop/Transit_Dashboard/data/census_tract_data/toronto_ct_centroids1.geojson",
    transport_network=transport_network,
    output_file="../results/travel_time_matrix_census_tract.csv",
    origin_id_col='CTUID',  # Specify the correct column name for origin ID if it's different
    destination_id_col='CTUID'  # Specify the correct column name for destination ID if it's different
)


Running time: 59.479373931884766 seconds


In [15]:
#  dissemination area to healthcare facilities 
compute_travel_time_matrix(
    origins_file="/Users/max/Desktop/Transit_Dashboard/data/census_tract_data/toronto_da_centroids.geojson",
    destinations_file="/Users/max/Desktop/Transit_Dashboard/data/key_destination_data/Healthcare.geojson",
    transport_network=transport_network,
    output_file="../results/travel_time_matrix_hospitals.csv",
    origin_id_col='DAUID',  # Specify the correct column name for origin ID if it's different
    destination_id_col='ADDRESS_POINT_ID'  # Specify the correct column name for destination ID if it's different
)

Exception ignored in: <function TransportNetwork.__del__ at 0x10d62ad30>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/r5py/r5/transport_network.py", line 99, in __del__
    self.osm_file.close()
TypeError: No matching overloads found for com.conveyal.osmlib.OSM.close(), options are:
	public void com.conveyal.osmlib.OSM.close()



Running time: 295.32969999313354 seconds


In [12]:
load_travel_time_matrix_census_tract = pd.read_csv("../results/travel_time_matrix_census_tract.csv")
display(load_travel_time_matrix_census_tract.head())

,Unnamed: 0,from_id,to_id,travel_time
0,0,5350128.04,5350128.04,0.0
1,1,5350128.04,5350363.06,81.0
2,2,5350128.04,5350363.07,90.0
3,3,5350128.04,5350378.23,90.0
4,4,5350128.04,5350378.24,94.0
